# LLM_1 - **10%** vs. LLM_2 - **90%**

In [1]:
import chess
from stockfish import Stockfish
from ollama import chat
from ollama import ChatResponse
import json
import random
import pandas as pd
import asyncio

In [2]:
# STOCKFISH_PATH = r"C:\Users\nafis\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"

STOCKFISH_ELO = 3500
NUM_GAMES = 100
MOVE_LIMIT = 300
RANDOM_START = 0
QWEN_US_AI = 10
QWEN_CN_AI = 90

In [3]:
stockfish = Stockfish()
stockfish.update_engine_parameters({"Threads": 4, "Hash": 4096})

In [4]:
qwen_cn_wins = 0
qwen_us_wins = 0
draws = 0

In [5]:
# def generate_random_number(seed:int) -> int:
#     # random.seed(seed)
#     return random.randint(1, 100)

In [6]:
def valid_move_to_string(board):
    legal_moves = [
        board.san(move) for move in board.legal_moves]
    
    if not legal_moves:
        return "No legal moves."
    else:
        return ", ".join(legal_moves)

In [7]:
def get_move_history(board: chess.Board) -> str:
    """Return a formatted move history string like '1. e4 e5 2. Nf3 d5'."""
    temp_board = chess.Board()  # Start from the initial position.
    move_history = []
    
    # Iterate over the moves in the board's move stack.
    for i, move in enumerate(board.move_stack):
        # Get the SAN for the move in the current temporary board.
        move_san = temp_board.san(move)
        # Push the move so that the board state is updated.
        temp_board.push(move)
        
        # For white moves (even index), start a new entry with move number.
        if i % 2 == 0:
            move_history.append(f"{i//2 + 1}. {move_san}")
        # For black moves, append to the last white move entry.
        else:
            move_history[-1] += f" {move_san}"
    
    # Join all entries with a space.
    return " ".join(move_history)

In [8]:
async def get_move_with_timeout(stockfish, timeout=15):
    """
    Run stockfish.get_best_move() in a separate thread and wait for a maximum of `timeout` seconds.
    If the call exceeds the timeout, return None.
    """
    try:
        # Use asyncio.to_thread to run the blocking call in a separate thread.
        move = await asyncio.wait_for(asyncio.to_thread(stockfish.get_best_move), timeout)
        return move
    except asyncio.TimeoutError:
        # move = stockfish.get_best_move_time(1000)
        print(f"Timeout: stockfish.get_best_move() took more than 15 seconds.")
        return None

In [9]:
def get_qwen_move(board:chess.Board, fen:str, temp=0.0):
    valid_moves = valid_move_to_string(board)

    move_history = get_move_history(board)
    
    response: ChatResponse = chat(model='qwen:32b', messages=[
        {
            'role': 'system',
            'content': 'You are a grand master chess player.'
        },
        {
            'role': 'user',
            'content': f"""You are playing chess and it is your turn. This is the current state of the game. Use this to work out where the pieces are on the board:

FEN: {fen}

The possible set of legal moves are: 

Legal Moves: {valid_moves}

+(You have to choose one from the provided list. Do not choose a move that is not in the list.)

The move history is: {move_history}.

Output the best move in SAN format to follow this position. Use the following single blob of JSON. Do not include any other information.
{{
    "san": "The move in SAN format",
}}"""}], options={"temperature": temp})
    
    try:
        parsed_json = json.loads(response.message.content)
        qwen_move = parsed_json["san"]
    except Exception as e:
        print(f"Wrong format given by Qwen. \nQwen Output: {response.message.content}")

    

    return qwen_move

In [10]:
def get_random_move(board:chess.Board, fen:str, seed: int = 101):
    legal_moves = [
        board.san(move)
        for move in board.legal_moves
    ]

    random.seed(seed)

    # print(random.choice(legal_moves))
    return random.choice(legal_moves)

    # print(legal_moves[0])

In [11]:
for game_number in range(1, NUM_GAMES + 1):
    
    move_counter = 0

    print(f"Starting game {game_number}...")

    board = chess.Board()

    while not board.is_game_over() and move_counter <= MOVE_LIMIT:
        if board.turn == chess.WHITE and not board.is_game_over():
            num = random.randint(1,100)
            
            if num <= QWEN_US_AI:
                stockfish.set_fen_position(board.fen())
                qwen_us_move = await get_move_with_timeout(stockfish)

                if qwen_us_move is None:
                    print("No move received. Reinitializing Stockfish.")
                    stockfish = Stockfish(parameters={"Threads": 4, "Hash": 4096})  # Reinitialize Stockfish for the next game.
                    break
            
            else:
                qwen_us_move = get_qwen_move(board, board.fen(), 2.0)

            try:
                board.push_san(qwen_us_move)
            except Exception as e:
                print(f"Invalid move by Qwen white. Move: {qwen_us_move}")
                qwen_us_move = get_random_move(board, board.fen(), 42)
                board.push_san(qwen_us_move)
                print(f"New Move: {qwen_us_move}")
                # continue
            
            move_counter += 1        
            print(f"Qwen US move: {qwen_us_move} Move number: {move_counter}")

        elif board.turn == chess.BLACK and not board.is_game_over():
            num = random.randint(1, 100)

            if num <= QWEN_CN_AI:
                stockfish.set_fen_position(board.fen())
                qwen_cn_move = await get_move_with_timeout(stockfish)

                if qwen_cn_move is None:
                    print("No move received. Reinitializing Stockfish.")
                    stockfish = Stockfish(parameters={"Threads": 4, "Hash": 4096})  # Reinitialize Stockfish for the next game.
                    break

            else:
                qwen_cn_move = get_qwen_move(board, board.fen(), 2.0)

            try:
                board.push_san(qwen_cn_move)
            except Exception as e:
                print(f"Invalid move by Qwen black. Move: {qwen_cn_move}")
                qwen_cn_move = get_random_move(board, board.fen(), 42)
                board.push_san(qwen_cn_move)
                print(f"New Move: {qwen_cn_move}")
                # continue
            move_counter += 1
            print(f"Qwen CN move: {qwen_cn_move} Move number: {move_counter}")   

    # Record the result of the game
    result = board.result()
    if result == "1-0":
        qwen_us_wins += 1
    elif result == "0-1":
        qwen_cn_wins += 1
    else:
        draws += 1

    print(f"Game {game_number} result: {result}\n\n")

Starting game 1...
Qwen US move: e4 Move number: 1
Qwen CN move: e5 Move number: 2
Qwen US move: Nf3 Move number: 3
Qwen CN move: b8c6 Move number: 4
Qwen US move: Nxe5 Move number: 5
Qwen CN move: c6e5 Move number: 6
Qwen US move: Nc3 Move number: 7
Qwen CN move: e5g6 Move number: 8
Invalid move by Qwen white. Move: Nxe5
New Move: f3
Qwen US move: f3 Move number: 9
Qwen CN move: c7c6 Move number: 10
Qwen US move: d2d4 Move number: 11
Qwen CN move: Nxe5 Move number: 12
Invalid move by Qwen white. Move: Nxe5
New Move: Bb5
Qwen US move: Bb5 Move number: 13
Qwen CN move: e5g6 Move number: 14
Qwen US move: Bxc6 Move number: 15
Qwen CN move: d7c6 Move number: 16
Qwen US move: Nd5 Move number: 17
Qwen CN move: c6d5 Move number: 18
Invalid move by Qwen white. Move: Nxb7
New Move: g3
Qwen US move: g3 Move number: 19
Qwen CN move: g8e7 Move number: 20
Qwen US move: d1e2 Move number: 21
Invalid move by Qwen black. Move: Qe4
New Move: h6
Qwen CN move: h6 Move number: 22
Qwen US move: Qd3 Move num

In [12]:
board.result()

'0-1'

In [13]:
results = []

In [14]:
results.append({
            "Qwen US Wins": qwen_us_wins,
            "Qwen CN Wins": qwen_cn_wins,
            "Draws": draws,
        })

print(results)

[{'Qwen US Wins': 0, 'Qwen CN Wins': 100, 'Draws': 0}]


In [15]:
df = pd.DataFrame(results)
df.to_csv("Qwen vs Qwen.csv", index=False)
print("Results saved to Qwen vs Qwen.csv")

Results saved to Qwen vs Qwen.csv
